<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/%EA%B0%81%EC%B8%B5_%EC%88%9C%EC%A0%84%ED%8C%8C%2C_%EC%97%AD%EC%A0%84%ED%8C%8C_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# 필요한 함수들

# Affine층의 순전파와 역전파

범용적인 affine 층 구현

In [ ]:
# 4차원 텐서를 염두해둔 코드이기 떄문에 다음 학기때완전히 이해가능
class Affine:
    def __init__(self, W, b):
        self.W = W # 가중치 행렬
        self.b = b # bias 벡터

        # 입력된 data 초기화
        self.x = None
        self.original_x_shape = None
        # 가중치와 bias 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1) # 4차원 텐서를 행렬로 변환. -1을 입력하면 reshape 가능한 숫자를 알아서 대입한다.
        self.x = x

        out = np.dot(self.x, self.W) + self.b # affine 변환. bias의 경우 배치처리를 할경우 행렬뎃섬에서 shape이 맞지 않는데 numpy에서 자동으로 벡터를 복사해 각 열에 더하게 된다.

        return out

    def backward(self, dout): # dout:흘러들어온 미분
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0) # 행렬의 경우 axis 0: 행, 1: 열. 텐서의 경우 axis 0: 각 행렬, 1: 각 행렬의 행, 2: 각 행렬의 열

        dx =dx.reshape(*self.original_x_shape) # 입력데이터

        return dx

In [ ]:
x = np.array([[1,1]])

W = np.array([[1,2,3],[4,5,6]])

b = np.array([7,8,9])

test = Affine(W,b)

In [ ]:
# 순전파 테스트
test.forward(x)

In [ ]:
# 역전파 테스트
print(test.backward(np.array([[2,1,-1]])))
print(test.dW)
print(test.db)

In [ ]:
# 배기처리 테스트
y = np.array([[1,1],[2,3]])

# 순전파 테스트
print(test.forward(y))

# 역전파 테스트
print(test.backward(np.array([[2,1,-1],[1,1,1]])))
print(test.dW)
print(test.db)

간단한 affine층 구현

In [ ]:
class Affine_t:
    def __init__(self, W, b): # W:가중치 matrix, b:bias
        self.W = W
        self.b = b

    def forward(self, x):
        self.x = x # x:data
        out = np.dot(self.x, self.W) + self.b # affine 변환

        return out

    def backward(self, dout): # dout:흘러들어온 미분
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0) # 덧셈노드의 역전파는 원래 흘러들어온 값이 그대로 통과하지만 배치처리를 할경우 계산을 위해 편향벡터를 배치크기만큼 반복한 행이 만들어져 행렬이 구성된다.
                                       # 이 경우 각열을 모두 더한 값이 역전파의 결과로 반환된다.(수학적인 증명은 아님.) 자세한 설명은 노트참고

        return dx

softmax 함수

In [ ]:
def softmax(x):
    if x.dim == 2:   # x가 행렬일 경우, 즉 배치처리를 해서 각 행이 확률벡터인 메트릭스를 입력받은 경우.
        x = x.T # 각 행이 확률벡터인 초기 메트릭스에서 각 열이 확률벡터인 메트릭스로 전치 시킨다.
        x = x - np.max(x, axis = 0) # np.max(x, axis = 0)는 각 열에 대한 최댓값을 저장한 리스트(벡터)를 반환 한다. 그리고 x에 대한 - 연산은 각 열의 확률벡터에 각 열의 최댓값을 빼주는 연산이다.(밑의 오버플로우 방지를 위한 최댓값 빼기와 같음.)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0) # 위와 같은 행렬과 벡터의 연산 원리.( 밑의 예제 참고.)
        return y.T

    x = x - np.max(x) # 오버플로를 방지하기위해 작성. 노트참고. , vector에 scalar를 빼면 numpy에서 알아서 원소별로 연산을 한다.
    return np.exp(x) / np.sum(np.exp(x))  # normalize

cross entropy 함수

In [ ]:
def corss_entropy_error(y, t):
    if y.ndim == 1: # y가 벡터인 경우, 배치처리를 안 한 경우
        t = t.reshape(1, t.size) # 10차원 벡터를 행렬로 변환.(개념상.)
        y = y.reshape(1, y.size)

    if t.size == y.size: # 라벨이 원-핫 인코딩이 돼어 있다면, 그러니까 입력 값이 10차원 벡터라면, size함수는 메트릭스의 원소갯수를 반환한다.
        t = t.argmax(axis=1) # 원-핫 인코딩 이전으로 되돌리기 위해 작성.

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size # 설명1, 확률벡터로 이루어진 행렬에서 알맞은 인덱스를 얻기위한 과정.

# Sigmoid층의 순전파와 역전파

sigmoid함수는 고전적인 미분값계산시 미분계산이 간단한 덧셈, 곱셈 문제로 바뀐다. 또한 계산그래프로도 이러한 결과를 도출할 수 있다.(노트 참고)

sigmoid함수의 역전파는

$\frac{\partial L}{\partial y}y(1-y)$

In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

# RELU층의 순전파와 역전파

RELU함수는 Deep Nueral Network에서 발생하는 sigmoid함수의 vanishing gradient문제를 해결할 수 있다.(노트 참고)

In [ ]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0) # self.mask는 0보다 크면 false 0보다 작으면 true인 리스트를 반환한다.
        out = x.copy()
        out[self.mask] = 0 # 설명1

        return out

    def backward(self, dout):
        dout[self.mask] = 0 # 위의 설명1을 참조. 주의할점은 흘러들어온 미분값의 부호에 의해 1이나 0을 곱하는것이 정해지는게 아니라 위의 x값을 기준으로 gradient를 살리거나 죽이게 된다.
        dx = dout

        return dx

테스트

In [ ]:
BTS = Relu()

print(BTS.forward(np.array([1,-2,3,-4])))
print(BTS.mask)
print(BTS.backward(np.array([1,2,-3,-4])))

설명

In [ ]:
# 설명1
test = np.array([1,2,3,4,])
test1 = np.array([True, False, False, False])
test[test1] = 0
test

# Softmax with Loss층의 순전파와 역전파
softmax층과 loss층은 특이하게도 따로 미분하는 것 보다 합성합수로 미분하는 것이 더 간단한 식을 도출할 수 있다. 고전적인 미분이나 계산 graph를 이용한 역전파로 구할 수 있는 미분계수는 뺄셈연산으로 간단하게 변한다.

$\frac{\partial L}{\partial a} = y-t$

In [ ]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None #손실함수
        self.y = None # softmax의 출력(확률 벡터)
        self.t = None # 정답 레이블(one-hot encoding 형태)

    def forward(self, x,  t):
        self.t = t
        self.y = softmax(x)
        self.loss = corss_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout = 1): # 역전파의 시작은 1 이다.
        batch_size = self.t.shape[0]

        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩이 돼어 있는 경우.
            dx = (self.y - self.t) / batch_size # 손실함수의 정의 값은 각각의 손실함수값의 평균이므로 배치사이즈로 나누어 준다.

        else: # 원-핫 이코딩이 돼어 있지 않은 경우.
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1 # 도출된 확률벡터는 배치처리로 묶여있다는 것을 생각해보면 위와 같은 결과를 도출함.
            dx = dx / batch_size

            return dx